# 06b: Calculate costs

In this script, we calculate the following cost estimates for regional averages of impacts

- low estimate (0.2 quantile)
- median cost
- damage costs
- prevention costs
- taxation costs
- budget constraint

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import brightway2 as bw

In [3]:
output_fp = "../output/" + BW_PROJECTNAME

In [4]:
mfs_perspectives_cc = xr.open_dataarray("../data/mfs_perspectives_extended_euro_{}.nc".format(EURO_REF_YEAR))
mfs_mc_sample_cc = xr.open_dataarray("../data/mfs_monte_carlo_sample_extended_euro{}.nc".format(str(EURO_REF_YEAR)))

We calculate costs for the regional averages.

In [5]:
def get_quantile_costs(impact_df, q, mfs, contract_dim):
    all_costs = xr.DataArray(
        np.tensordot(impact_df[list(mfs.coords["LCIA method"].values)].to_numpy(),
                    mfs.to_numpy(),
                    axes=([1,], [contract_dim,])),
        coords={
            "tech index": list(range(len(impact_df))),
            "impact category": list(mfs.coords["impact category"].values),
            "sample index": list(range(mfs.shape[1]))
        }
    )

    a = all_costs.to_numpy()
    qcosts = np.where(np.all(a < 0, axis=2),
                      np.quantile(a, 1-q, axis=2),
                      np.quantile(a, q, axis=2))

    return pd.DataFrame(qcosts,
                        index=impact_df.index,
                        columns=list(mfs.coords["impact category"].values))

def get_perspectives_cost(impact_df, mfs):
    df_mfs = mfs_perspectives_cc.to_dataframe().reset_index().pivot(
        index=["perspective", "impact category"],
        columns=["LCIA method"],
        values="value")
    
    return pd.DataFrame(
        np.tensordot(impact_df[df_mfs.columns].to_numpy(), df_mfs.to_numpy().T, axes=1),
        index=impact_df.index,
        columns=df_mfs.index
    )


### Full impacts

In [6]:
impacts = pd.read_csv(output_fp+"/impacts_battery_storage.csv").set_index(
    ["ecoinvent name", "scenario", "year", "location"])

low_costs = get_quantile_costs(impacts, 0.2, mfs_mc_sample_cc, 2)
median_costs = get_quantile_costs(impacts, 0.5, mfs_mc_sample_cc, 2)

low_costs.reset_index(inplace=True)
median_costs.reset_index(inplace=True)

low_costs["perspective"] = "low estimate"
median_costs["perspective"] = "median"

perspectives_costs = get_perspectives_cost(impacts, mfs_perspectives_cc)

perspectives_costs = perspectives_costs.melt(ignore_index=False).reset_index().pivot(
    columns="impact category", values="value",
    index=["ecoinvent name", "scenario", "year", "location", "perspective"])

costs_all_perspectives = pd.concat(
    (
        low_costs,
        median_costs,
        perspectives_costs.reset_index()
    ),
    axis=0
).set_index(["ecoinvent name", "scenario", "year", "location", "perspective"]).sort_index()

costs_all_perspectives.to_csv(output_fp+"/costs_battery_storage.csv")

## Get capacity ratios and capacity factors

In [15]:
def read_mif(fp):
    return pd.read_csv(fp, sep=";").iloc[:, :-1]

def get_ratios(mif, var1, var2, years, s1=1.0, s2=1.0):
    sel = mif[["Variable", "Region"]+[str(y) for y in years]]

    df1 = sel[sel["Variable"] == var1].set_index(["Region"]).drop(columns="Variable").astype(float)
    df2 = sel[sel["Variable"] == var2].set_index(["Region"]).drop(columns="Variable").astype(float)

    return (s1 * df1) / (s2 * df2)

def select_var(mif, var, years):
    sel = mif[["Variable", "Region"]+[str(y) for y in years]]
    df1 = sel[sel["Variable"] == var].set_index(["Region"]).drop(columns="Variable").astype(float)

    return df1



In [8]:
storage_vars = [
    "New Cap|Electricity|Storage|Battery|For PV",
    "New Cap|Electricity|Storage|Battery|For Wind",
]

elec_vars = [
    "New Cap|Electricity|Solar|+|PV",
    "New Cap|Electricity|Wind|+|Onshore",
]

techs = [
    "PV commercial",
    "wind, onshore, medium",
]

dflist = []
for scen in SCENARIOS_BATTERIES:
    mif = read_mif("../data/REMIND_generic_{}.mif".format(scen))
    
    for tech, v1, v2 in zip(techs, storage_vars, elec_vars):
        df = get_ratios(mif, v1, v2, YEARS).reset_index()
        df["scenario"] = scen
        df["short name"] = tech
        dflist.append(df)

df = pd.concat(dflist).rename(columns={"Region": "region"}).set_index(["short name", "scenario", "region"])
cap_ratios = df.melt(ignore_index=False, var_name="year", value_name="capacity ratio")

cap_ratios.reset_index().to_csv(output_fp + "/capacity_ratios_battery_storage_new_cap.csv", index=False)


In [9]:
storage_vars = [
    "Cap|Electricity|Storage|Battery|For PV",
    "Cap|Electricity|Storage|Battery|For Wind",
]

elec_vars = [
    "Cap|Electricity|Solar|+|PV",
    "Cap|Electricity|Wind|+|Onshore",
]

techs = [
    "PV commercial",
    "wind, onshore, medium",
]

dflist = []
for scen in SCENARIOS_BATTERIES:
    mif = read_mif("../data/REMIND_generic_{}.mif".format(scen))
    
    for tech, v1, v2 in zip(techs, storage_vars, elec_vars):
        df = get_ratios(mif, v1, v2, YEARS).reset_index()
        df["scenario"] = scen
        df["short name"] = tech
        dflist.append(df)

df = pd.concat(dflist).rename(columns={"Region": "region"}).set_index(["short name", "scenario", "region"])
cap_ratios = df.melt(ignore_index=False, var_name="year", value_name="capacity ratio")

cap_ratios.reset_index().to_csv(output_fp + "/capacity_ratios_battery_storage_total_cap.csv", index=False)

In [10]:
production_vars = [
    "SE|Electricity|Solar|+|PV",
    "SE|Electricity|Wind|+|Onshore",
]

cap_vars = [
    "Cap|Electricity|Solar|+|PV",
    "Cap|Electricity|Wind|+|Onshore",
]

techs = [
    "PV commercial",
    "wind, onshore, medium",
]

# scaling factors
scap = 1e-03 # from giga to tera-watt
sprod = 31.71e-03 # EJ to TWa

dflist = []
for scen in SCENARIOS_BATTERIES:
    mif = read_mif("../data/REMIND_generic_{}.mif".format(scen))
    
    for tech, vprod, vcap in zip(techs, production_vars, cap_vars):
        df = get_ratios(mif, vprod, vcap, YEARS, s1=sprod, s2=scap).reset_index()
        df["scenario"] = scen
        df["short name"] = tech
        dflist.append(df)

df = pd.concat(dflist).rename(columns={"Region": "region"}).set_index(["short name", "scenario", "region"])
cap_ratios = df.melt(ignore_index=False, var_name="year", value_name="capacity factor")

cap_ratios.reset_index().to_csv(output_fp + "/capacity_factors_VRE.csv", index=False)

## Get shares of wind vs solar PV in VRE mixes by region

In [11]:
bw.projects.set_current(BW_PROJECTNAME)

In [12]:
def get_dbname(scen, year):
    matches = [db for db in bw.databases if "{}_{}".format(scen, year) in db]

    return matches[0]

In [13]:
dflist = []
for scen in SCENARIOS:
    for year in YEARS:
        dbname = get_dbname(scen, year)
        db = bw.Database(dbname)
        matches = [act for act in db if act["name"] == "market group for electricity, high voltage, VRE inputs"]
        data = {}
        for act in matches:
            onshore_input = 0
            PV_input = 0
            for exc in act.technosphere():
                input = exc.input["name"]
                if "photovoltaic" in input:
                    PV_input += exc.amount
                if "wind" in input and "onshore" in input:
                    onshore_input += exc.amount
            data[act["location"]] = {"PV commercial": PV_input, "wind, onshore, medium": onshore_input}
        df = pd.DataFrame(data)
        df.index.name = "short name"
        df = df.reset_index()
        df["scenario"] = scen
        df["year"] = year
        dflist.append(df)

        print(f"{scen}, {year} done.")


SSP2-NPi, 2020 done.
SSP2-NPi, 2030 done.
SSP2-NPi, 2040 done.
SSP2-NPi, 2050 done.
SSP2-PkBudg500, 2020 done.
SSP2-PkBudg500, 2030 done.
SSP2-PkBudg500, 2040 done.
SSP2-PkBudg500, 2050 done.


In [14]:
pd.concat(dflist).melt(id_vars=["scenario", "year", "short name"], value_vars=REMIND_REGIONS,
                       var_name="region", value_name="share").to_csv(output_fp+"/vre_input_shares.csv", index=False)

## Get PEM efficiencies

In [19]:
dflist = []
for scen in SCENARIOS_BATTERIES:
    mif = read_mif("../data/REMIND_generic_{}.mif".format(scen))

    df = select_var(mif, "Tech|Hydrogen|Electricity|Efficiency", YEARS).reset_index()
    df["scenario"] = scen
    dflist.append(df)

df = pd.concat(dflist).rename(columns={"Region": "region"}).set_index(["scenario", "region"])
efficiencies = df.melt(ignore_index=False, var_name="year", value_name="PEM efficiency")
efficiencies.reset_index().to_csv(output_fp + "/PEM_efficiencies.csv", index=False)
